## 58 , 59 : location_latitude , location_longitude

In [1]:
#[1] Imports
import pandas as pd
import numpy as np
from pyproj import Transformer

In [2]:
# [2] Load data
train = pd.read_csv("/mnt/e/bootcamp/Project/Divar.csv")
train_copy = train.copy()

/tmp/ipykernel_152253/1412100633.py:2: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("/mnt/e/bootcamp/Project/Divar.csv")


In [3]:
#[3] name columns
x_longitude = train_copy["location_longitude"]
y_latitude  = train_copy["location_latitude"]

In [4]:
# [4] transformer (WGS84 -> UTM Zone 39N)
transformer = Transformer.from_crs("epsg:4326", "epsg:32639", always_xy=True)

utm_easting, utm_northing = transformer.transform(
    x_longitude.values, y_latitude.values
)
# adding new columns
train_copy["UTM_Easting"]  = utm_easting
train_copy["UTM_Northing"] = utm_northing

In [5]:
# [5] imputing by neighborhood_slug
utm_df = train_copy.copy()

neigh_means = utm_df.groupby("neighborhood_slug")[["UTM_Easting","UTM_Northing"]].transform("mean")
utm_df["UTM_Easting"]  = utm_df["UTM_Easting"].fillna(neigh_means["UTM_Easting"])
utm_df["UTM_Northing"] = utm_df["UTM_Northing"].fillna(neigh_means["UTM_Northing"])

print(utm_df[["UTM_Easting","UTM_Northing"]].isna().sum())


UTM_Easting     199755
UTM_Northing    199755
dtype: int64


In [6]:
# [6] imputing by city_slug
city_means = utm_df.groupby("city_slug")[["UTM_Easting","UTM_Northing"]].transform("mean")

utm_df["UTM_Easting"]  = utm_df["UTM_Easting"].fillna(city_means["UTM_Easting"])
utm_df["UTM_Northing"] = utm_df["UTM_Northing"].fillna(city_means["UTM_Northing"])

print(utm_df[["UTM_Easting","UTM_Northing"]].isna().sum())

UTM_Easting     2
UTM_Northing    2
dtype: int64


In [8]:
# [7] imputing two remaind rows
i_shiraz = 676502
i_tehran = 292674

shiraz_mean = utm_df.loc[utm_df["city_slug"]=="shiraz", ["UTM_Easting","UTM_Northing"]].mean()
tehran_mean = utm_df.loc[utm_df["city_slug"]=="tehran", ["UTM_Easting","UTM_Northing"]].mean()

utm_df.loc[i_shiraz, ["UTM_Easting","UTM_Northing"]] = shiraz_mean
utm_df.loc[i_tehran, ["UTM_Easting","UTM_Northing"]] = tehran_mean

print(utm_df[["UTM_Easting","UTM_Northing"]].isna().sum())

UTM_Easting     0
UTM_Northing    0
dtype: int64


In [9]:
# [8] Replacing columns in train_copy
train_copy["UTM_Easting"]  = utm_df["UTM_Easting"].values
train_copy["UTM_Northing"] = utm_df["UTM_Northing"].values

##